In [1]:
import os
import pandas as pd
import pyreadstat
import glob
from functools import wraps
from time import time

In [2]:
#Además de los decimales, del word, saca los valores que sirven la checkear que la carga es correcta
#Automatizando eso en una función, if True, then print('los datos están bien')
#Pide a Laura que te pase las tablas dinámicas
#INtenta construir una serie temporal para análisis
#Reproduce los análisis que han hecho en el WORD del estudio

In [17]:
def timer(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print('Elapsed time: {}'.format(end-start))
        return result
    return wrapper

#Funciones GASTOS
@timer
def get_tablas_gastos(base_dir):
    for file in os.listdir(base_dir):
        file_path = f'{base_dir}/{file}'
        df, meta = pyreadstat.read_sav(file_path)
            
        df = df.apply(lambda x: x.astype(float)/100 if x.name not in ['ANOENC', 'NUMERO','CODIGO', 'NACTOR'] else x)
        df = df.apply(lambda x: x.astype(int) if x.name in ['ANOENC', 'NUMERO'] else x)
        df = df.apply(lambda x: x/1000000 if x.name in ['NACTOR'] else x)
        df.to_csv(base_dir+'/gastos_'+ str(df.iloc[0][0]) + '.csv')        

@timer
def load_tablas_gastos(base_dir):
    all_files = glob.glob(base_dir + "/*.csv")
    
    li = []
    for fname in all_files:
        df = pd.read_csv(fname, index_col=None, header=0)
        li.append(df)

    dframe = pd.concat(li, axis=0, ignore_index=True) 
    dframe.to_csv('gastos_serie_completa.csv')
      

#Funciones HOGAR
@timer
def get_tablas_hogar(base_dir):
    for file in os.listdir(base_dir):
        file_path = f'{base_dir}/{file}'
        df, meta = pyreadstat.read_sav(file_path)
        df.to_csv(base_dir+'/hogar_'+ str(df.loc[0, "ANOENC"]) + '.csv')

@timer
def load_tablas_hogar(base_dir):
    all_files = glob.glob(base_dir + "/*.csv")
    
    li = []
    for fname in all_files:
        df = pd.read_csv(fname, index_col=None, header=0)
        li.append(df)

    dframe = pd.concat(li, axis=0, ignore_index=True) 
    dframe.to_csv('hogar_serie_completa.csv')

def validity(df_gastos, df_hogar):
    return df_gastos.shape == df_hogar.shape
    
#Funciones MIEMBROS

In [5]:
ruta = "/home/fserrey/gitrep/ACA_project/docs_table/gastos_tablas"

get_tablas_gastos(ruta)

Elapsed time: 115.03039336204529


In [6]:
load_tablas_gastos(ruta)

Elapsed time: 90.64820194244385


In [7]:
gastos_tabla = pd.read_csv('gastos_serie_completa.csv')

In [8]:
gastos_tabla.head(2)

,Unnamed: 0,Unnamed: 0.1,ANOENC,NUMERO,CODIGO,GASTO,PORCEN,CANTIDAD,GASTMON,GASTNOM1,GASTNOM2,GASTNOM3,GASTNOM4,GASTNOM5,NACTOR
0,0,0,2009,1,1112,267539.46,0.0,92240.99,267539.46,NaN,NaN,NaN,NaN,NaN,707.602898
1,1,1,2009,1,1113,102409.64,0.0,29517.12,102409.64,NaN,NaN,NaN,NaN,NaN,707.602898


In [9]:
gastos_tabla.ANOENC.value_counts()

2010    2195196
2009    2177881
2008    2171403
2007    2127839
2006    1677656
Name: ANOENC, dtype: int64

In [10]:
ruta = "/home/fserrey/gitrep/ACA_project/docs_table/hogar_tablas"

get_tablas_hogar(ruta)

Elapsed time: 9.257530212402344


In [11]:
load_tablas_hogar(ruta)

Elapsed time: 10.214112520217896


In [12]:
hogar_tabla = pd.read_csv('hogar_serie_completa.csv')

In [13]:
hogar_tabla.head(2)

,Unnamed: 0,Unnamed: 0.1,ANOENC,NUMERO,CCAA,NUTS1,CAPROV,TAMAMU,DENSI,CLAVE,...,IMPEXAC,IMPUEXAC,INTERIN,IMPU_INTER,NUMPERI,COMIMH,COMISD,COMIHU,COMIINV,COMITOT
0,0,0,2009,1,8,4,6,5,3,2,...,745.0,1,2,6,1.0,84.0,0.0,0.0,8.0,92.0
1,1,1,2009,2,10,5,6,5,3,1,...,1122.0,6,3,6,1.0,56.0,0.0,0.0,3.0,59.0


In [14]:
hogar_tabla.ANOENC.value_counts()

2009    22346
2010    22203
2008    22077
2007    21542
2006    19435
Name: ANOENC, dtype: int64

In [18]:
validity(gastos_tabla, hogar_tabla)

False

In [16]:
gastos_tabla.shape

(10349975, 15)

In [19]:
hogar_tabla.shape

(107603, 176)